```
Задание 1

Основным модулем для работы с регулярными выражениями в питоне является re. Но есть ещё активно развивающийся regex. Найдите одно любое преимущество regex и опишите его, сопроводив примером (текст, на котором вы демонстрируете отличие, должен быть русским). Пользоваться можно любыми ресурсами, начните поиск с ссылок в семинарской тетрадке. 
```

In [306]:
import re, regex

Можно сматчить частично.

In [307]:
pattern = regex.compile(r'(ОШИБКА)')

In [308]:
pattern.match('ОШИБ', partial=True)

<regex.Match object; span=(0, 4), match='ОШИБ', partial=True>

In [309]:
pattern.match('ОШИБ', partial=True).partial

True

In [310]:
pattern.match('ОШИБКА', partial=True).partial

False

Отдельный функционал, чтобы сматчить строку целиком.

In [311]:
print(pattern.fullmatch('НЕ ОШИБКА'))

None


In [312]:
pattern.fullmatch('ОШИБКА')

<regex.Match object; span=(0, 6), match='ОШИБКА'>

In [313]:
print(pattern.fullmatch(' ОШИБКА '))

None


Очень удобная штука, которая позволяет использовать f-строки:

In [314]:
regex.subf(r'(\w+) (\w+)', 'Это второе слово "{2}", а это первое "{1}"', "раз два")

'Это второе слово "два", а это первое "раз"'

И нечеткий матчинг, само собой.

In [315]:
pattern = regex.compile(r'(?:ОШИБКА){i}') # лишние символы

In [316]:
pattern.match('ДОБШИБКА')

<regex.Match object; span=(0, 8), match='ДОБШИБКА', fuzzy_counts=(0, 2, 0)>

In [317]:
pattern = regex.compile(r'(?:ОШИБКА){e}') # ошибки

In [318]:
pattern.match('АШИПКА')

<regex.Match object; span=(0, 6), match='АШИПКА', fuzzy_counts=(2, 0, 0)>

In [319]:
pattern = regex.compile(r'(?:ОШИБКА){d}') # недостающие символы

In [320]:
pattern.match('ОШБКА')

<regex.Match object; span=(0, 5), match='ОШБКА', fuzzy_counts=(0, 0, 1)>

Можно использовать, чтобы находить в тексте слова, даже если в них есть ошибки. Пример тривиальный, но этого достаточно, чтобы продемонстрировать функционал.

In [321]:
silly_text = "Мидвежата старше всех. Они радились еще холодной зимой в берлоги."

In [322]:
pattern1 = regex.compile(r'(?:МЕДВЕЖАТА){e<=2}', flags=regex.IGNORECASE)
pattern2 = regex.compile(r'(?:родились){e<=2}', flags=regex.IGNORECASE)

In [323]:
pattern1.findall(silly_text)

['Мидвежата']

In [324]:
pattern2.findall(silly_text)

[' радились']

Теоретически такие же задачи можно решить и с помощью модуля ```re```, но на это потребуется существенно больше времени.

```
Задание 2

Скачайте вот этот текст - http://gramota.ru/class/coach/punct/45_177

Приведите его к нижнему регистру, удалите все знаки пунктуации, токенизируйте (любым способом). После этого сделайте стемминг и найдите по несколько примеров на каждый из видов ошибок:

    1) два разных слова ошибочно свелись к одинаковой основе
    2) две словоформы одного слова после стемминга отличаются друг от друга
    3) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)

```

In [325]:
with open('gramota.txt', encoding='utf-8') as file: # скопировал в файл, чтобы не забивать тетрадку
    gramota = file.read()

In [326]:
gramota = gramota.replace('\n', ' ').lower()

In [327]:
gramota[:259]

'выберите правильные варианты ответов. для проверки выполненного задания нажмите кнопку «проверить».   обособление обстоятельств обстоятельство – второстепенный член предложения, который обозначает признак действия или другого признака. обстоятельства поясняют'

In [328]:
import nltk

In [329]:
gramota = nltk.word_tokenize(gramota)

In [330]:
gramota = [word.strip('_') for word in gramota if word.isalpha()]

In [331]:
gramota[:5]

['выберите', 'правильные', 'варианты', 'ответов', 'для']

In [332]:
stemmer = nltk.stem.snowball.SnowballStemmer('russian')

In [333]:
stems = [(word, stemmer.stem(word)) for word in gramota]

#####  1) два разных слова ошибочно свелись к одинаковой основе

In [334]:
only_stems = set([stem for word, stem in stems])

In [335]:
len(stems), len(only_stems) #основ очевидно меньше

(846, 425)

In [336]:
stems_reverse = {}
for key, value in stems:
    stems_reverse.setdefault(value, set()).add(key)

In [337]:
[(key, value) for key, value in stems_reverse.items() if len(value) > 1][:10]

[('обстоятельств', {'обстоятельств', 'обстоятельства', 'обстоятельство'}),
 ('член', {'член', 'члены'}),
 ('предложен', {'предложении', 'предложения', 'предложениях'}),
 ('котор', {'которые', 'который'}),
 ('признак', {'признак', 'признака'}),
 ('друг', {'другие', 'другого'}),
 ('сказуем', {'сказуемые', 'сказуемым'}),
 ('выделя', {'выделяется', 'выделять', 'выделяются'}),
 ('случа', {'случае', 'случаи', 'случай', 'случаях'}),
 ('рассмотр', {'рассмотрим', 'рассмотрите'})]

Например,
<ul>
    <li>('вид', {'вид', 'видит', 'видов'})</li>
    <li>('выражен', {'выражением', 'выраженные'})</li>
    <li> ('цел', {'цели', 'целый', 'целью'})</li>
</ul>

По-моему, ошибок не так много, учитывая ограничения метода. Не во всех случаях вообще можно без контекста оценить ошибочность.

##### 2) две словоформы одного слова после стемминга отличаются друг от друга

In [338]:
stems[:10]

[('выберите', 'выбер'),
 ('правильные', 'правильн'),
 ('варианты', 'вариант'),
 ('ответов', 'ответ'),
 ('для', 'для'),
 ('проверки', 'проверк'),
 ('выполненного', 'выполнен'),
 ('задания', 'задан'),
 ('нажмите', 'нажм'),
 ('кнопку', 'кнопк')]

In [339]:
set([(word, stem) for word, stem in stems if 'ча' in word])

{('деепричастием', 'деепричаст'),
 ('деепричастий', 'деепричаст'),
 ('деепричастным', 'деепричастн'),
 ('молча', 'молч'),
 ('обозначает', 'обознача'),
 ('отвечают', 'отвеча'),
 ('отвечающие', 'отвеча'),
 ('отличать', 'отлича'),
 ('примечаний', 'примечан'),
 ('скучал', 'скуча'),
 ('случае', 'случа'),
 ('случаи', 'случа'),
 ('случай', 'случа'),
 ('случая', 'случ'),
 ('случаях', 'случа'),
 ('торчал', 'торча'),
 ('часов', 'час'),
 ('частями', 'част'),
 ('чаще', 'чащ')}

Здесь тоже не очень много примеров.
<ul>
    <li>('сидел', 'сидел') – ('сидя', 'сид')</li>
    <li>('случая', 'случ') – ('случаях', 'случа')</li>
</ul>

##### 3) слово не изменилось после стемминга

In [340]:
case3 = set([stem for stem in stems if len(stem[0]) > 4 and stem[0] == stem[1]])

In [341]:
case3 

{('будет', 'будет'),
 ('вдруг', 'вдруг'),
 ('взмахнув', 'взмахнув'),
 ('видел', 'видел'),
 ('воннегут', 'воннегут'),
 ('вопрос', 'вопрос'),
 ('встает', 'встает'),
 ('выйдет', 'выйдет'),
 ('вынырнул', 'вынырнул'),
 ('город', 'город'),
 ('дворник', 'дворник'),
 ('имеет', 'имеет'),
 ('могут', 'могут'),
 ('может', 'может'),
 ('мороз', 'мороз'),
 ('например', 'например'),
 ('оборот', 'оборот'),
 ('обстоятельств', 'обстоятельств'),
 ('перед', 'перед'),
 ('подряд', 'подряд'),
 ('пошел', 'пошел'),
 ('признак', 'признак'),
 ('пушкин', 'пушкин'),
 ('сидел', 'сидел'),
 ('сотрудник', 'сотрудник'),
 ('сумел', 'сумел'),
 ('хотел', 'хотел'),
 ('через', 'через')}

```
Задание 3
Проанализируйте список стоп-слов из нлтк (для русского). Какие ещё слова вы бы туда добавили? (5 слов будет достаточно, не забудьте аргументировать ваш выбор)
```

In [342]:
nltk_stopwords = nltk.corpus.stopwords.words('russian')

Список состоит из наречий, местоимений, частиц, но он, очевидно, не полный.

In [344]:
nltk_stopwords[:20]

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты']

Cтоит проанализировать покрытие какого-нибудь текста имеющимися стоп-словами:

In [345]:
with open('war_and_peace.txt', encoding='utf-8') as file:
    war_and_peace = file.read()

In [346]:
war_and_peace = nltk.word_tokenize(war_and_peace)

In [347]:
war_and_peace = [word.lower() for word in war_and_peace if word.isalpha()]

Для этого посмотрим на наиболее частотные слова, уже отфильтрованные по списку:

In [348]:
import collections

In [349]:
war_and_peace = collections.Counter(war_and_peace).most_common()

In [350]:
candidates = [word for word, freq in war_and_peace if word not in nltk_stopwords]

In [351]:
candidates[:10]

['это',
 'сказал',
 'пьер',
 'князь',
 'сноска',
 'наташа',
 'андрей',
 'время',
 'сказала',
 'которые']

Полученный результат содержит много частиц, местоимений и местоимений-прилагательных. Если слова в тексте рассматривать изолированно, то их можно отнести к неинформативным. Разные формы последних составляют существенный объём текста. Предлагаю добавить их, но всё же отмечу, что не в каждой задаче это будет правильными решением.

In [352]:
suggestion = ('это', 'который', 'этих', 'тех', 'свой')

In [353]:
for word in suggestion: print(word in nltk_stopwords)

False
False
False
False
False


Дополним имеющийся список

In [354]:
nltk_stopwords.extend(suggestion)

```
Задание 4

Скачайте вот этот текст - https://zona.media/news/2019/09/11/zhukov-expertise

Предобработайте текст двумя способами:
1) Разбейте текст на предложения sent_tokenize из nltk, токенизируйте каждое предложение с помощью регулярного выражения (попробуйте разные и выберите лучшее), лемматизируйте токены с помощью pymorphy2

2) предобработайте текст с помощью mystem3 

Ответьте на вопросы:
1) Что лучше разбивает на предложения (mystem или sent_tokenize)?
2) Что лучше токенизирует (mystem или ваша регулярка)? Если mystem, то можно ли улучшить регулярное выражение так, чтобы качество было выше?
3) Что лучше для лемматизации mystem или pymorphy?

Важно, чтобы ответы на вопросы были аргументированы. 
```

*pymorphy2 + nltk + re*

In [355]:
with open('zhuk.txt', encoding='utf-8') as file:
    zhuk = file.read()

In [356]:
zhuk = zhuk.replace('\n', ' ').replace('\u200b', ' ')

In [357]:
zhuk_pym = nltk.sent_tokenize(zhuk)

In [358]:
pattern = r'([А-Яа-яA-Za-z]+-*[А-Яа-я]*-*[А-Яа-я]*)'

In [359]:
words_re = []
for sent in zhuk_pym:
    words_re.extend(re.findall(pattern, sent))

In [360]:
words_re[-5:]

['перевел', 'студента', 'под', 'домашний', 'арест']

In [361]:
import pymorphy2

In [362]:
pymorph = pymorphy2.MorphAnalyzer()

In [363]:
words_pym = [pymorph.parse(word)[0][2] for word in words_re]

In [364]:
words_pym[-5:]

['перевести', 'студент', 'под', 'домашний', 'арест']

*mystem3*

In [365]:
import pymystem3

In [366]:
mystem = pymystem3.Mystem()

In [367]:
import json

Файл получен путём обработки текста бинарным файлом, поскольку доступа к функционалу с разбивкой на предложения через Python  нет
```
./mystem -iscd --format json zhuk.txt zhuk_sent.txt
```

In [368]:
with open('zhuk_sent.txt', encoding='utf-8') as file:
    zhuk_sent = [json.loads(line) for line in file]

Разбиваем на предложения

In [369]:
sents_mys = []
for paragraph in zhuk_sent:
    current = [word['text'] for word in paragraph]
    current = ''.join(current).strip('\r\n').split('\\s')
    sents_mys.extend(current)

Поскольку лемматизация уже проведёна и доступна в самом файле, получим эти данные:

In [370]:
lemmas_mys = []
for paragraph in zhuk_sent:
    lemmas_mys.extend([word['analysis'][0]['lex'] for word in paragraph if 'analysis' in word.keys() and word['analysis']])

In [371]:
lemmas_mys[-5:]

['переводить', 'студент', 'под', 'домашний', 'арест']

В обоих случаях анализтор не понимает, что текст, не отделённый точкой или другим знаком препинания, завершающим предложение, но отделённый знаком переноса строки, в начале файла может быть его заголовком. Пришлось его убрать для чистоты эксперимента.

```
1) Что лучше разбивает на предложения (mystem или sent_tokenize)?
```

По предложениям получаем одинаковую разбивку :

In [375]:
zhuk_pym[-1] == sents_mys[-1]

True

In [376]:
len(zhuk_pym) == len(sents_mys)

True

Разница только в том, что на вариант с mystem пришлось потрать больше времени и сил, поэтому мой голос в пользу **sent_tokenize**

```
2) Что лучше токенизирует (mystem или ваша регулярка)? Если mystem, то можно ли улучшить регулярное выражение так, чтобы качество было выше?
```

Здесь опять же получился одинаковый результат. Регулярка вполне адекватно справилась, написать её было проще, чем разобраться в многоуровневой вложенности списков и словарей mystem.

Единственный минус регулярки оказался в том, в результате потерялся "yotube" из "youtube-канал". Улучшить её можно, добавив и латиницу.

In [377]:
words_re[-9:]

['По',
 'ходатайстве',
 'следствие',
 'суд',
 'перевел',
 'студента',
 'под',
 'домашний',
 'арест']

In [378]:
sents_mys[-1] #предложение собрано из отдельных токенов

'По ходатайстве следствие суд перевел студента под домашний арест.'

```
3) Что лучше для лемматизации mystem или pymorphy?
```

Результаты получились схожие, за исключением:

* pymorphy предпочитает совершенный вид, а mystem несовершенный
* единственное упущение pymorphy – "сизый", а не "сизо"
* малозначительное разделение "о" и "об"
* pymorphy выделяет "она" как лемму "её"

In [379]:
list(zip(words_pym, lemmas_mys))[-5:]

[('перевести', 'переводить'),
 ('студент', 'студент'),
 ('под', 'под'),
 ('домашний', 'домашний'),
 ('арест', 'арест')]

Здесь мой голос в пользу **pymorphy**, т.к. использовать его было проще.

Резюмируя, можно сказать, что на данном тексте действительно получаются довольно близкие результаты, но это не значит, что на каком-то другом тексте у mystem не будет бы преимущества, при этом будет ли это преимущество оправдывать неудобство использования – отдельный вопрос.